# Hidden Markov Models for Internet Measurements Analysis

**RESCOM 2019 • [Sandrine Vaton](http://perso.telecom-bretagne.eu/sandrinevaton/) (IMT Atlantique)**  
**Modèles de chaînes de Markov cachées pour l’analyse de traces sur Internet**  

In this interactive notebook we will compare different statistical models for RTT (round-trip time) time series observed on the Internet.  
We will highlight the limitations of classical parametric models, and show that nonparametric Bayesian approaches allow to obtain a segmentation close to what a human expert would achieve.

We will use RTT time series from the [RIPE Atlas](https://atlas.ripe.net/) Internet measurement infrastructure.  
We will use the [scikit-learn](https://scikit-learn.org/stable/) library for mixture models, [hmmlearn](https://hmmlearn.readthedocs.io/en/latest/) for parametric HMMs, and the [Atlas Trends API](https://labs.ripe.net/Members/maxime_mouchet/api-for-summarising-events-in-ripe-atlas-rtt-time-series) for nonparametric HMMs.

#### Contents

1. [Introduction](#Introduction)
2. [Parametric Models](#Parametric-Models)  
    2.1 [Mixture Model](#Mixture-Model)  
    2.2 [Hidden Markov Model](#Hidden-Markov-Model)
3. [Nonparametric Models](#Nonparametric-Models)  
    3.1 [Dirichlet Process Mixture Model](#Dirichlet-Process-Mixture-Model)  
    3.2 [Hierarchical Dirichlet Process Hidden Markov Model](#Hierarchical-Dirichlet-Process-Hidden-Markov-Model)

In [ ]:
try:
    import google.colab, sys
    !git clone https://github.com/RESCOM19-HMM/Lab.git
    !pip install hmmlearn
    sys.path.append('Lab')
except:
    import sys; sys.path.append('..')

In [ ]:
import pandas as pd
from tqdm.auto import tqdm

In order to keep things simple in the notebook, the code for statistical inference and plotting is wrapped in the
[trends](https://github.com/RESCOM19-HMM/Lab/blob/master/trends) and [utils](https://github.com/RESCOM19-HMM/Lab/blob/master/utils) modules.

In [ ]:
from trends import *
from utils import *

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set_style('whitegrid')

## Introduction
_([Contents](#Contents))_

In RIPE Atlas, a measurement (ping, traceroute, ...) is performed by one or more probes towards a single destination.  
A destination can be any device reachable on the Internet (another probe, a server, a router, ...).  
An origin-destination pair is defined by the couple (measurement ID, probe ID).  
In this notebook we will consider measurement between anchors, which are probes with more computational resources.

In [ ]:
client = AtlasTrendsClient(verbose=True)

In [ ]:
# Measurement: Ping IPv4 for anchor at-vie-as1120.anchors.atlas.ripe.net
# Probe: us-bos-as26167.anchors.atlas.ripe.net 
at_vie_us_bos = dict(
    msm_id   = 1437285,                   # Atlas measurement ID
    prb_id   = 6222,                      # Atlas probe ID
    start_dt = utc_datetime(2018, 5, 2),  # (Optional) Default: stop date - 7 days
    stop_dt  = utc_datetime(2018, 5, 10), # (Optional) Default: the current date
)

# Measurement: Ping IPv4 for anchor at-vie-as1120.anchors.atlas.ripe.net
# Probe: us-chi-as2914.anchors.atlas.ripe.net
at_vie_us_chi = dict(
    msm_id   = 1437285,
    prb_id   = 6343, 
    start_dt = utc_datetime(2018, 5, 2),
    stop_dt  = utc_datetime(2018, 5, 10),
)

# Change this line to choose an origin-destination pair
query = at_vie_us_bos

In [ ]:
# Fetch the RTT time series from the RIPE Atlas API
df = client.fetch_ticks(**query, as_df=True) # `as_df` returns a Pandas DataFrame instead of a JSON object

In [ ]:
plt.figure(figsize=(16,2.5))
plot_rtt(df)

In [ ]:
# Scikit-learn does not handle missing values;
# they are however supported by the Atlas Trends API.
X = df.rtt.fillna(method='ffill')[:,np.newaxis]

## Parametric Models
_([Contents](#Contents))_

Model                                              | Number of states | Time dependency
:--------------------------------------------------|:-----------------|:--------------
Mixture Model                                      | Fixed            | No
Hidden Markov Model                                | Fixed            | Yes
Dirichlet Process Mixture Model                    | $\infty$         | No
Hierarchical Dirichlet Process Hidden Markov Model | $\infty$         | Yes

Inference is performed using the EM algorithm. Initialized with a k-means.

In [ ]:
from utils.models import GaussianMixture, GaussianHMM

### Mixture Model

In [ ]:
model = GaussianMixture.from_samples(X, n_components=10)
print('EM stopped after {} iterations'.format(model.n_iter_))
print('Final log-likelihood = {}'.format(model.log_likelihood(X)))

In [ ]:
plt.figure(figsize=(16,2.5))
plot_rtt(df)
plot_sequence(df.rtt.index, model.predict(X))

In [ ]:
fit_interactive(df, GaussianMixture)

#### Choice of the number of components

https://en.wikipedia.org/wiki/Bayesian_information_criterion

In [ ]:
def fit_penalized(X, model_class, n_components):
    stats, models = [], []
    for k in tqdm(n_components):
        model = model_class.from_samples(X, k)
        models.append(model)
        stats.append({
            'aic': model.aic(X),
            'bic': model.bic(X),
            'log_likelihood': model.log_likelihood(X)
        })
    stats = pd.DataFrame.from_records(stats, index=n_components)
    best = stats.bic.values.argmin()
    return n_components[best], models[best], stats

In [ ]:
n_components = range(2,20)
best_n, best_model, stats = fit_penalized(X, GaussianMixture, n_components)
print('Best number of components = {}'.format(best_n))

In [ ]:
plot_penalized_likelihood(stats)

In [ ]:
plt.figure(figsize=(16,2.5))
plot_rtt(df)
plot_sequence(df.rtt.index, best_model.predict(X))

### Hidden Markov Model

In [ ]:
fit_interactive(df, GaussianHMM)

#### Choice of the number of components

In [ ]:
n_components = range(2, 10)
best_n, best_model, stats = fit_penalized(X, GaussianHMM, n_components)
print('Best number of components = {}'.format(best_n))

In [ ]:
plot_penalized_likelihood(stats)

In [ ]:
plt.figure(figsize=(16,2.5))
plot_rtt(df)
plot_sequence(df.rtt.index, best_model.predict(X))

## Nonparametric Models
_([Contents](#Contents))_

In [ ]:
from utils.models import BayesianGaussianMixture

### Dirichlet Process Mixture Model

In [ ]:
model = BayesianGaussianMixture.from_samples(X, 10)

In [ ]:
print('Inference stopped after {} iterations'.format(model.n_iter_))
print('Number of states inferred = {}'.format(model.n_states(X)))
print('Final log-likelihood = {}'.format(model.log_likelihood(X)))

In [ ]:
plt.figure(figsize=(16,2.5))
plot_rtt(df)
plot_sequence(df.rtt.index, model.predict(X))

### Hierarchical Dirichlet Process Hidden Markov Model

In [ ]:
df = client.fetch_trends(**query, as_df=True)

In [ ]:
plt.figure(figsize=(16,2.5))
plot_trends(df)

In [ ]:
states = sorted(df.state.unique())
_, axes = plt.subplots(nrows=2, ncols=int(len(states)/2), gridspec_kw={'hspace': 0.5}, figsize=(16,5))
for ax, state in zip(axes.ravel(), states):    
    plot_kde(df, ax=ax, states=[state])